In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



This notebook is in response to https://www.kaggle.com/c/tabular-playground-series-oct-2021/discussion/280986
It is not supposed to a notebook dedicated to modelling data for the TPS Oct 



From the docmentation  https://contrib.scikit-learn.org/category_encoders/targetencoder.html

"For the case of categorical target: features are replaced with a blend of posterior probability of the target given particular categorical value and the prior probability of the target over all the training data."

But we know that the correlation of features to target is very low with this dataset as ALL the EDA (and SHAP) notebooks came to the same conclusion. Actually this makes sense for what we see TargetEncoding actually do. 
The new values it comes up with are very close to 0.5 suggesting that correlation of feature to target is very very low.


So what was a sequence of 0's and 1's now becomes a sequence of 0.4999 and 0.5001
Either way it is simply 2 possible values , which my head still reads binary and as such it should not make any difference to the scores. But it is since the LB score is definetly changing.

Wait. The golden rule is trust in CV. I need to check and see how the CV score is changing.

Without target encoding www.kaggle.com/mohammadkashifunique/tsp-single-xgboost-model it is 
Overall Validation Score: 0.8572129946017066
With target encoding  www.kaggle.com/mohammadkashifunique/single-xgboost-model-featureengineering it is  
Overall Validation Score: 0.8572199414082429

That seems very close. A bit of a change in the 6th decimal place of CV score is producing a 2 point change in the 5th decimal place in the LB score.
That is food for thought!! Is the 30% of scoring data being used somehow biased on any boolean features.

I might need to think about this point some more...
Also I don't trust the caluclation of the CV score becuase it is based on the average over the 5 folds and not calculated once on the entire set of OOF precitions.

For now though we need to expand on exactly what is changing when we run TargetEncoding. At the most basic level we are changing the variance of that feature. More specifically of every feature that we run TargetEncoding on. 
So a quick recap on variance.

Variance is a measure of dispersion, meaning it is a measure of how far a set of numbers are spread out from their average value. 

In [ ]:
np_joe1 = [0, 0, 0, 0, 0, 1, 1, 1, 1]
print ("Var joe1 : ", np.var(np_joe1))
np_joe2 = [0.49, 0.49, 0.49, 0.49, 0.49, 0.51, 0.51, 0.51, 0.51]
print ("Var joe2: ", np.var(np_joe2))



Now I don't know all the technical details of how each model interacts with the variance of an indivual feature, but I do know that they do.

From the StandardScaler documenation http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
    
    Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual features do not more or 
    less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance).
    
The StandardScaler sorts this out by "Standardize features by removing the mean and scaling to unit variance."


Reading ths above again, I'm thinkning that it is not that easy to change a bimodal distribution to a Gaussian one. So model sensitivity is likely to be more on variance and less on dsitribution. 



So to test this theory I have modified my copy of the original notebook 
https://www.kaggle.com/joecooper/tsp-single-xgboost-model

The difference between v1 and v3 is mainly the inclusion of the standard scaler. It largely confirms what the above is saying : having reduced or even better, having 'managed' variance for features will improve the models estimators and performance.